In [1]:
import mxnet as mx
from os import path as osp
import glob
import re
import scipy.io as sio
from mxnet import autograd, gluon, image, init, nd
import numpy as np
from matplotlib import pyplot as plt
import cv2
from mxnet.gluon.data import DataLoader
from mxnet.gluon.data import Dataset
from mxnet.image import imread
from mxnet.gluon import nn,model_zoo
import time
from utils.transforms import TrainTransform, TestTransform
from trainers import reidEvaluator, reidTrainer



In [2]:

class Market1501(object):

    dataset_dir = 'Market-1501-v15.09.15'

    def __init__(self, root='', **kwargs):
        self.dataset_dir = osp.join(root, self.dataset_dir)
        self.train_dir = osp.join(self.dataset_dir, 'bounding_box_train')
        self.query_dir = osp.join(self.dataset_dir, 'query')
        self.gallery_dir = osp.join(self.dataset_dir, 'bounding_box_test')

        self._check_before_run()

        train, num_train_pids, num_train_imgs = self._process_dir(self.train_dir, relabel=True)
        query, num_query_pids, num_query_imgs = self._process_dir(self.query_dir, relabel=False)
        gallery, num_gallery_pids, num_gallery_imgs = self._process_dir(self.gallery_dir, relabel=False)
        num_total_pids = num_train_pids + num_query_pids
        num_total_imgs = num_train_imgs + num_query_imgs + num_gallery_imgs

        print("=> Market1501 loaded")
        print("Dataset statistics:")
        print("  ------------------------------")
        print("  subset   | # ids | # images")
        print("  ------------------------------")
        print("  train    | {:5d} | {:8d}".format(num_train_pids, num_train_imgs))
        print("  query    | {:5d} | {:8d}".format(num_query_pids, num_query_imgs))
        print("  gallery  | {:5d} | {:8d}".format(num_gallery_pids, num_gallery_imgs))
        print("  ------------------------------")
        print("  total    | {:5d} | {:8d}".format(num_total_pids, num_total_imgs))
        print("  ------------------------------")

        self.train = train
        self.query = query
        self.gallery = gallery

        self.num_train_pids = num_train_pids
        self.num_query_pids = num_query_pids
        self.num_gallery_pids = num_gallery_pids

    def _check_before_run(self):
        """Check if all files are available before going deeper"""
        if not osp.exists(self.dataset_dir):
            raise RuntimeError("'{}' is not available".format(self.dataset_dir))
        if not osp.exists(self.train_dir):
            raise RuntimeError("'{}' is not available".format(self.train_dir))
        if not osp.exists(self.query_dir):
            raise RuntimeError("'{}' is not available".format(self.query_dir))
        if not osp.exists(self.gallery_dir):
            raise RuntimeError("'{}' is not available".format(self.gallery_dir))

    def _process_dir(self, dir_path, relabel=False):
        img_paths = glob.glob(osp.join(dir_path, '*.jpg'))
        pattern = re.compile(r'([-\d]+)_c(\d)')

        pid_container = set()
        for img_path in img_paths:
            pid, _ = map(int, pattern.search(img_path).groups())
            if pid == -1: continue  # junk images are just ignored
            pid_container.add(pid)
        pid2label = {pid: label for label, pid in enumerate(pid_container)}

        dataset = []
        for img_path in img_paths:
            pid, camid = map(int, pattern.search(img_path).groups())
            if pid == -1:
                continue  # junk images are just ignored
            assert 0 <= pid <= 1501  # pid == 0 means background
            assert 1 <= camid <= 6
            camid -= 1  # index starts from 0
            if relabel: pid = pid2label[pid]
            dataset.append((img_path, pid, camid))

        num_pids = len(pid_container)
        num_imgs = len(dataset)
        return dataset, num_pids, num_imgs



__factory = {
    'market': Market1501
}


def get_names():
    return __factory.keys()


def init_dataset(name, *args, **kwargs):
    if name not in __factory.keys():
        raise KeyError("Unknown datasets: {}".format(name))
    return __factory[name](*args, **kwargs)


dataset=Market1501()

=> Market1501 loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # images
  ------------------------------
  train    |   751 |    12936
  query    |   750 |     3368
  gallery  |   751 |    15913
  ------------------------------
  total    |  1501 |    32217
  ------------------------------


In [3]:
from collections import defaultdict

import numpy as np
from mxnet.gluon.data.sampler import Sampler


class RandomIdentitySampler(Sampler):
    def __init__(self, data_source, num_instances=4):
        self.data_source = data_source#train_img
        self.num_instances = num_instances
        self.index_dic = defaultdict(list)
        for index, (_, pid,_) in enumerate(data_source):
            self.index_dic[pid].append(index)#第几个是当前类别的
        self.pids = list(self.index_dic.keys())#类别编号
        self.num_identities = len(self.pids)#类别总数

    def __iter__(self):
        indices = np.random.permutation(self.num_identities)
        ret = []
        for i in indices:
            pid = self.pids[i]
            t = self.index_dic[pid]
            replace = False if len(t) >= self.num_instances else True
            t = np.random.choice(t, size=self.num_instances, replace=replace)
            ret.extend(t)
        return iter(ret)

    def __len__(self):
        return self.num_identities * self.num_instances#751*4
    
class ImageData(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, item):
        img, pid, camid = self.dataset[item]
        img = imread(img)
        if self.transform is not None:
            img = self.transform(img)
        return img, pid, camid

    def __len__(self):
        return len(self.dataset)

    
# import random

# import mxnet as mx
# from mxnet.gluon.data.vision import transforms as T


# class Random2DTranslation(object):
#     """
#     With a probability, first increase image size to (1 + 1/8), and then perform random crop.

#     Args:
#         height (int): target height.
#         width (int): target width.
#         p (float): probability of performing this transformation. Default: 0.5.
#     """

#     def __init__(self, height, width, p=0.5, interpolation=1):
#         self.height = height
#         self.width = width
#         self.p = p
#         self.interpolation = interpolation

#     def __call__(self, img):
#         """
#         Args:
#             img (mx.image): Image to be cropped.

#         Returns:
#             PIL Image: Cropped image.
#         """
#         if random.random() < self.p:
#             return T.Resize((self.width, self.height), interpolation=self.interpolation)(img)
#         new_width, new_height = int(
#             round(self.width * 1.125)), int(round(self.height * 1.125))
#         resized_img = T.Resize((new_width, new_height), interpolation=self.interpolation)(img)
#         x_maxrange = new_width - self.width
#         y_maxrange = new_height - self.height
#         x1 = int(round(random.uniform(0, x_maxrange)))
#         y1 = int(round(random.uniform(0, y_maxrange)))
#         croped_img = mx.image.fixed_crop(resized_img, x1, y1, self.width, self.height)
#         return croped_img


# class TrainTransform(object):
#     def __init__(self, h, w):
#         self.h = h
#         self.w = w

#     def __call__(self, x):
#         x = Random2DTranslation(self.h, self.w)(x)
#         x = T.RandomFlipLeftRight()(x)
#         x = T.RandomColorJitter(brightness=0.4, contrast=0.4,
#                                               saturation=0.4)(x)
#         x = T.ToTensor()(x)
#         x = T.Normalize(mean=(0.485, 0.456, 0.406),
#                         std=(0.229, 0.224, 0.225))(x)
# #         x = T.Cast(dtype='float16')(x)
#         return x


# class TestTransform(object):
#     def __init__(self, h, w):
#         self.h = h
#         self.w = w

#     def __call__(self, x=None):
#         x = T.Resize((self.w, self.h))(x)
#         x = T.ToTensor()(x)
#         x = T.Normalize(mean=(0.485, 0.456, 0.406),
#                         std=(0.229, 0.224, 0.225))(x)
# #         x = T.Cast(dtype='float16')(x)
#         return x



In [4]:
height=224
width=224
# height = 256
# width = 128
num_instances = 4
train_batch=32
workers=0
test_batch=32
trainloader = DataLoader(#有23组来自由sampler组成3004张图片（751*4）
            ImageData(dataset.train, TrainTransform(height, width)),#路径
            sampler=RandomIdentitySampler(dataset.train, num_instances),
            batch_size=train_batch, num_workers=workers, last_batch='discard'
        )
queryloader = DataLoader(
        ImageData(dataset.query, TestTransform(height, width)),
        batch_size=test_batch, num_workers=workers,
    )

galleryloader = DataLoader(
    ImageData(dataset.gallery, TestTransform(height, width)),
    batch_size=test_batch, num_workers=workers,
)

In [5]:
ctx=mx.gpu(0)

class resnet_multi(nn.HybridBlock):
    def __init__(self, num_label, **kwargs):
        super(resnet_multi, self).__init__()
        self.num_label = num_label
        self.kwargs = kwargs
        with self.name_scope():
            self.models = self.net()
            
    def hybrid_forward(self,F, X):
#         if self.kwargs.get("train_angle", False):
        public_block, classifier_block = self.models
#         public_block, classifier_block,score_block = self.models
#         else:
#         public_block, landmark_block = self.models
        global_feat = public_block(X)
#         global_feat = score_block(global_feat)
        score=classifier_block(global_feat)
#         if self.kwargs.get("train_angle", False):
        return score, global_feat
#         else:
#             return landmark_block(x)

    def net(self):
        model = nn.HybridSequential()
        # public block
        finetune_net = model_zoo.vision.resnet50_v2(pretrained=True,ctx=ctx)
#         finetune_net = model_zoo.vision.densenet121(pretrained=True,ctx=ctx)#效果最好


        model.add(finetune_net.features)#model 一个输出
#         model.cast("float16")
        # classifier_block
        classifier_block = nn.HybridSequential()
        with classifier_block.name_scope():
            classifier_block.add(
                nn.Dense(512,weight_initializer=mx.initializer.MSRAPrelu('out', 0)),
#                 nn.Dense(2048,activation="relu"),
                nn.BatchNorm(),
                nn.LeakyReLU(0.1),
                nn.Dropout(0.5),
                nn.Dense(self.num_label, weight_initializer=mx.initializer.Normal(0.001))
            )
#         classifier_block.hybridize()    
        classifier_block.initialize(ctx=ctx)
#         classifier_block = classifier_block.cast('float16') 
        
    
#         classifier_block.collect_params()
    
#         classifier_block.collect_params().setattr("lr_mult",10)
        model.add(classifier_block)

        return model
    

model = resnet_multi(num_label=dataset.num_train_pids)
 
# net.set
# print (net1())
# model.
model.hybridize()
# net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
nd.waitall()
# save as symbol
data =mx.sym.var('data')

sym = model(data)

img=nd.random_uniform(shape=(1,3,224,224),ctx=ctx)
print(model(img))
print(model)

[
[[ 1.23904003e-02 -1.15005430e-02  9.12461663e-04 -1.96854491e-02
  -2.42906669e-03  3.28694214e-03  6.01742119e-02  2.30051414e-03
  -4.88901744e-03 -3.76654826e-02  2.63874773e-02 -8.22955277e-03
   1.57312173e-02 -1.71848964e-02 -2.63790810e-03 -9.45427641e-03
  -4.39939415e-03  2.67034732e-02  7.95619003e-03 -4.46429923e-02
  -2.55427044e-02  3.98282483e-02  1.82303786e-02 -1.76996086e-03
  -2.29514427e-02 -1.51555408e-02  1.93009488e-02  5.53213852e-03
  -2.94753574e-02  1.28040640e-02  2.33905180e-03  2.54036835e-03
   4.88458341e-03  2.57647969e-02 -1.07256917e-03  2.45175716e-02
   2.05023084e-02 -2.49082409e-03 -4.76065185e-03  3.28411460e-02
   2.86135897e-02  3.64177153e-02  2.18072850e-02  2.04847474e-02
   1.07200455e-03 -1.53825963e-02 -1.08306324e-02 -6.56667724e-03
  -2.26747319e-02 -7.06921285e-03  2.15645712e-02 -2.16826773e-03
  -8.68983939e-03 -1.71095114e-02 -1.42747350e-02  2.79766340e-02
  -2.20943168e-02  1.23463422e-02 -6.36728946e-04  4.95214164e-02
   2.412

In [7]:
#loss
# summary_writer = SummaryWriter(osp.join(save_dir, 'tensorboard_log'))



class TripletLoss(object):#如何定义loss 
    """Modified from Tong Xiao's open-reid (https://github.com/Cysu/open-reid).
    Related Triplet Loss theory can be found in paper 'In Defense of the Triplet
    Loss for Person Re-Identification'."""

    def __init__(self, margin=None):
        self.margin = margin

    def __call__(self, global_feat, labels, normalize_feature=False):#（2048,751）
        if normalize_feature:
            global_feat = normalize(global_feat, axis=-1)
        dist_mat = euclidean_dist(global_feat, global_feat)
        dist_ap, dist_an = hard_example_mining(
            dist_mat, labels)
        loss = nd.relu(dist_ap - dist_an + self.margin)
#         loss = nd.max(dist_ap - dist_an+self.margin,0)
        return loss, dist_ap, dist_an


def euclidean_dist(x, y):#计算欧式距离，输入二位矩阵
    m, n = x.shape[0], y.shape[0]
    xx = nd.power(x, 2).sum(axis=1, keepdims=True).broadcast_to((m, n))
    yy = nd.power(y, 2).sum(axis=1, keepdims=True).broadcast_to((n, m)).T
    dist = xx + yy
    dist = dist - 2 * nd.dot(x, y.T)
    dist = dist.clip(a_min=1e-12, a_max=1e12).sqrt()  # for numerical stability
    return dist




def hard_example_mining(dist_mat, labels, return_inds=False):

    assert len(dist_mat.shape) == 2
    assert dist_mat.shape[0] == dist_mat.shape[1]
    N = dist_mat.shape[0]

    # shape [N, N]
    is_pos = nd.equal(labels.broadcast_to((N, N)), labels.broadcast_to((N, N)).T).astype('float32')
    is_neg = nd.not_equal(labels.broadcast_to((N, N)), labels.broadcast_to((N, N)).T).astype('float32')
    # `dist_ap` means distance(anchor, positive)
    # both `dist_ap` and `relative_p_inds` with shape [N, 1]
    dist_pos = dist_mat * is_pos
    dist_ap = nd.max(dist_pos, axis=1)
    # `dist_an` means distance(anchor, negative)
    # both `dist_an` and `relative_n_inds` with shape [N, 1]
    dist_neg = dist_mat * is_neg + nd.max(dist_mat, axis=1, keepdims=True) * is_pos
    dist_an = nd.min(dist_neg, axis=1)
    return dist_ap, dist_an



def hard_example_mining(dist_mat, labels, return_inds=False):

    assert len(dist_mat.shape) == 2
    assert dist_mat.shape[0] == dist_mat.shape[1]
    N = dist_mat.shape[0]

    # shape [N, N]
    is_pos = nd.equal(labels.broadcast_to((N, N)), labels.broadcast_to((N, N)).T).astype('float32')
    is_neg = nd.not_equal(labels.broadcast_to((N, N)), labels.broadcast_to((N, N)).T).astype('float32')
    # `dist_ap` means distance(anchor, positive)
    # both `dist_ap` and `relative_p_inds` with shape [N, 1]
    dist_pos = dist_mat * is_pos
    dist_ap = nd.max(dist_pos, axis=1)
    # `dist_an` means distance(anchor, negative)
    # both `dist_an` and `relative_n_inds` with shape [N, 1]
    dist_neg = dist_mat * is_neg + nd.max(dist_mat*10000, axis=1, keepdims=True) * is_pos
    dist_an = nd.min(dist_neg, axis=1)
    dist_an=dist_an*is_neg + nd.max(dist_mat, axis=1, keepdims=True) * is_pos
    return dist_ap, dist_an




def cls_tri_criterion(cls_scores, feat, targets):
        cls_loss = xent_criterion(cls_scores, targets)
#         print(cls_loss)
        triplet_loss, dist_ap, dist_an = tri_criterion(feat, targets)
#         print(triplet_loss)
        loss = cls_loss + triplet_loss
        return loss



In [8]:
margin = 0.3
lr = 3e-4
weight_decay = 5e-4

xent_criterion = gluon.loss.SoftmaxCrossEntropyLoss()
# penalty=gluon.loss.SoftmaxCrossEntropyLoss()
tri_criterion = TripletLoss(margin)
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': lr, 'wd': weight_decay})
# reid_evaluator = reidEvaluator(model, mx.gpu(0))

In [9]:


def eval_func( distmat, q_pids, g_pids, q_camids, g_camids, max_rank=50):
    """Evaluation with market1501 metric
        Key: for each query identity, its gallery images from the same camera view are discarded.
        """
    num_q, num_g = distmat.shape
    if num_g < max_rank:
        max_rank = num_g
        print("Note: number of gallery samples is quite small, got {}".format(num_g))
    indices = np.argsort(distmat, axis=1)
    matches = (g_pids[indices] == q_pids[:, np.newaxis]).astype(np.int32)

    # compute cmc curve for each query
    all_cmc = []
    all_AP = []
    num_valid_q = 0.  # number of valid query
    for q_idx in range(num_q):
#         start=time.time()
        # get query pid and camid
        q_pid = q_pids[q_idx]
        q_camid = q_camids[q_idx]

        # remove gallery samples that have the same pid and camid with query
        order = indices[q_idx]
        remove = (g_pids[order] == q_pid) & (g_camids[order] == q_camid)
        keep = np.invert(remove)

        # compute cmc curve
        # binary vector, positions with value 1 are correct matches
        orig_cmc = matches[q_idx][keep]
        if not np.any(orig_cmc):
            # this condition is true when query identity does not appear in gallery
            continue

        cmc = orig_cmc.cumsum()
        cmc[cmc > 1] = 1

        all_cmc.append(cmc[:max_rank])
        num_valid_q += 1.

        # compute average precision
        # reference: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Average_precision
        num_rel = orig_cmc.sum()
        tmp_cmc = orig_cmc.cumsum()
        tmp_cmc = [x / (i + 1.) for i, x in enumerate(tmp_cmc)]
        tmp_cmc = np.asarray(tmp_cmc) * orig_cmc
        AP = tmp_cmc.sum() / num_rel
        all_AP.append(AP)
#         print (time.time()-start)

    assert num_valid_q > 0, "Error: all query identities do not appear in gallery"

    all_cmc = np.asarray(all_cmc).astype(np.float32)
    all_cmc = all_cmc.sum(0) / num_valid_q
    mAP = np.mean(all_AP)

    return all_cmc, mAP

In [10]:
import math
def adjust_lr(optimizer, ep):
    if ep < 20:
        lr = 1e-4 * (ep + 1) / 2
    else:
        lr=((1+math.cos(ep*3.14159/500))*0.001)/2
    optimizer.set_learning_rate(lr)

    
reid_evaluator = reidEvaluator(model, mx.gpu(0))

best_rank1 = -np.inf
best_epoch = 0
step_size = 40
    
for e in range(401,500):
    if step_size > 0:
        adjust_lr(trainer, e + 1)
    
    start = time.time()
    for i, (data, label,cameraid) in enumerate(trainloader):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with mx.autograd.record():
            score,feat = model (data)
            loss = cls_tri_criterion(score,feat,label)

        loss.backward()
        trainer.step(train_batch)
        curr_loss = nd.mean(loss).asscalar()
        
    if e>0 and e%10==0:
        rank1 = reid_evaluator.evaluate(queryloader, galleryloader)
        is_best=True
        is_best = rank1 > best_rank1

        if is_best:
            best_rank1 = rank1
            print("saving")
            model_name = './resnet/jxx_res'
            model.export(model_name,epoch=e)

    print("Epoch %s. Loss: %s,time_cost %.1f,learning rate %.9f" % (e, curr_loss,time.time()-start,trainer.learning_rate))

   
    
    

Epoch 0. Loss: 8.299433,time_cost 72.7,learning rate 0.000100000
Epoch 1. Loss: 7.6636844,time_cost 70.1,learning rate 0.000150000
Epoch 2. Loss: 7.0091133,time_cost 74.5,learning rate 0.000200000
Epoch 3. Loss: 6.3619266,time_cost 62.9,learning rate 0.000250000
Epoch 4. Loss: 6.1242085,time_cost 50.7,learning rate 0.000300000
Epoch 5. Loss: 5.776878,time_cost 49.8,learning rate 0.000350000
Epoch 6. Loss: 5.3655157,time_cost 52.8,learning rate 0.000400000
Epoch 7. Loss: 5.0444856,time_cost 49.0,learning rate 0.000450000
Epoch 8. Loss: 3.9540887,time_cost 44.1,learning rate 0.000500000
Epoch 9. Loss: 4.594185,time_cost 47.2,learning rate 0.000550000
Extracted features for query set, obtained 3368-by-1024 matrix
Extracted features for gallery set, obtained 15913-by-1024 matrix
Computing distance matrix
Computing CMC and mAP
Results ----------
mAP: 48.8%
CMC curve
Rank-1  : 71.1%
Rank-5  : 87.4%
Rank-10 : 91.8%
Rank-20 : 94.9%
------------------
saving
Epoch 10. Loss: 3.8616269,time_cost 

Epoch 86. Loss: 1.2158445,time_cost 41.2,learning rate 0.000887748
Epoch 87. Loss: 0.8454983,time_cost 42.2,learning rate 0.000885257
Epoch 88. Loss: 0.2600849,time_cost 42.7,learning rate 0.000882742
Epoch 89. Loss: 0.404794,time_cost 43.8,learning rate 0.000880203
Extracted features for query set, obtained 3368-by-1024 matrix
Extracted features for gallery set, obtained 15913-by-1024 matrix
Computing distance matrix
Computing CMC and mAP
Results ----------
mAP: 61.6%
CMC curve
Rank-1  : 82.5%
Rank-5  : 94.1%
Rank-10 : 95.8%
Rank-20 : 97.4%
------------------
Epoch 90. Loss: 0.30254915,time_cost 178.0,learning rate 0.000877641
Epoch 91. Loss: 0.6681838,time_cost 41.0,learning rate 0.000875056
Epoch 92. Loss: 0.46301913,time_cost 40.8,learning rate 0.000872447
Epoch 93. Loss: 0.23959751,time_cost 41.5,learning rate 0.000869816
Epoch 94. Loss: 0.33699343,time_cost 40.8,learning rate 0.000867161
Epoch 95. Loss: 0.890301,time_cost 41.2,learning rate 0.000864485
Epoch 96. Loss: 0.48333618,

Extracted features for query set, obtained 3368-by-1024 matrix
Extracted features for gallery set, obtained 15913-by-1024 matrix
Computing distance matrix
Computing CMC and mAP
Results ----------
mAP: 65.0%
CMC curve
Rank-1  : 85.3%
Rank-5  : 94.4%
Rank-10 : 96.3%
Rank-20 : 97.5%
------------------
Epoch 170. Loss: 0.17451622,time_cost 176.2,learning rate 0.000612901
Epoch 171. Loss: 0.103495754,time_cost 40.7,learning rate 0.000609072
Epoch 172. Loss: 0.053293277,time_cost 41.4,learning rate 0.000605236
Epoch 173. Loss: 0.11449635,time_cost 40.7,learning rate 0.000601394
Epoch 174. Loss: 0.29071698,time_cost 40.9,learning rate 0.000597546
Epoch 175. Loss: 0.044524744,time_cost 40.8,learning rate 0.000593691
Epoch 176. Loss: 0.14893909,time_cost 41.5,learning rate 0.000589831
Epoch 177. Loss: 0.008942661,time_cost 41.1,learning rate 0.000585965
Epoch 178. Loss: 0.22570498,time_cost 40.9,learning rate 0.000582094
Epoch 179. Loss: 0.22522253,time_cost 41.4,learning rate 0.000578218
Extra

Epoch 251. Loss: 0.022772044,time_cost 41.9,learning rate 0.000301427
Epoch 252. Loss: 0.33285308,time_cost 41.7,learning rate 0.000297829
Epoch 253. Loss: 0.0028777362,time_cost 41.1,learning rate 0.000294244
Epoch 254. Loss: 0.008048929,time_cost 41.2,learning rate 0.000290671
Epoch 255. Loss: 0.021030486,time_cost 41.6,learning rate 0.000287111
Epoch 256. Loss: 0.044407714,time_cost 41.2,learning rate 0.000283564
Epoch 257. Loss: 0.046278656,time_cost 41.3,learning rate 0.000280031
Epoch 258. Loss: 0.07494865,time_cost 41.0,learning rate 0.000276511
Epoch 259. Loss: 0.043024573,time_cost 40.9,learning rate 0.000273006
Extracted features for query set, obtained 3368-by-1024 matrix
Extracted features for gallery set, obtained 15913-by-1024 matrix
Computing distance matrix
Computing CMC and mAP
Results ----------
mAP: 69.5%
CMC curve
Rank-1  : 87.4%
Rank-5  : 95.7%
Rank-10 : 97.4%
Rank-20 : 98.4%
------------------
Epoch 260. Loss: 0.016832346,time_cost 179.0,learning rate 0.000269514


Epoch 333. Loss: 0.013872086,time_cost 42.6,learning rate 0.000065685
Epoch 334. Loss: 0.00082303886,time_cost 41.8,learning rate 0.000063753
Epoch 335. Loss: 0.0040766275,time_cost 42.3,learning rate 0.000061847
Epoch 336. Loss: 0.0013519902,time_cost 42.9,learning rate 0.000059969
Epoch 337. Loss: 0.052926604,time_cost 41.3,learning rate 0.000058118
Epoch 338. Loss: 0.002778082,time_cost 41.8,learning rate 0.000056294
Epoch 339. Loss: 0.004410627,time_cost 41.3,learning rate 0.000054497
Extracted features for query set, obtained 3368-by-1024 matrix
Extracted features for gallery set, obtained 15913-by-1024 matrix
Computing distance matrix
Computing CMC and mAP
Results ----------
mAP: 72.9%
CMC curve
Rank-1  : 90.0%
Rank-5  : 96.4%
Rank-10 : 97.7%
Rank-20 : 98.8%
------------------
saving
Epoch 340. Loss: 0.000837522,time_cost 176.7,learning rate 0.000052728
Epoch 341. Loss: 0.0146038905,time_cost 41.5,learning rate 0.000050987
Epoch 342. Loss: 0.0019374593,time_cost 40.3,learning rat

In [ ]:
e=10
if e>0 and e%10==0:
    print(1)

In [11]:
reid_evaluator.evaluate(queryloader, galleryloader)

Extracted features for query set, obtained 3368-by-1024 matrix
Extracted features for gallery set, obtained 15913-by-1024 matrix
Computing distance matrix
Computing CMC and mAP
Results ----------
mAP: 73.8%
CMC curve
Rank-1  : 89.7%
Rank-5  : 96.4%
Rank-10 : 98.0%
Rank-20 : 98.8%
------------------


0.8966746

In [6]:
model_name = './resnet/jxx_res'
model.export(model_name,epoch=0)
